In [2]:
pip install MeCab

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\Users\zoo58\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [3]:
import MeCab
import os
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
def searchFiles(path):
    filelist = []
    filenames = os.listdir(path)
    for filename in filenames:
        file_path = os.path.join(path, filename)
        filelist.append(file_path)
    return filelist
    
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
    tags = ['NNG','NNP','VV','VA','VX','VCP','VCN', 'MAG']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0])>1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos


In [5]:
reviews = []
for filePath in  searchFiles('../Data/'):
    review = pd.read_csv(filePath, encoding = 'utf-8')
    reviews.append(review)
docs = pd.concat(reviews, ignore_index=True)
print(docs)

              date press category  \
0       2020/01/01  서울신문    국제,정치   
1       2020/01/01  국민일보       정치   
2       2020/01/01  국민일보       정치   
3       2020/01/01  국민일보       경제   
4       2020/01/01  서울신문       정치   
...            ...   ...      ...   
316384  2020/10/31  세계일보       정치   
316385  2020/10/31  세계일보       정치   
316386  2020/10/31  서울신문       국제   
316387  2020/10/31  세계일보    사회,경제   
316388  2020/10/31  세계일보       국제   

                                             headline  \
0                            “교육이 계층이동 사다리 돼야 양극화 해소”   
1                     국군 전사자 12만3000여명, 아직도 못 돌아오고 있다   
2                 한국당의 처참한 성적표, “총사퇴는 쇼” “장기판 박카스 뚜껑”   
3                  쌀값·외식비·학원비는 올랐는데, 소비자물가 상승률은 역대 최저   
4                    “블루오션 잡아라”… 민주·정의, 청소년 당원 모시기 경쟁   
...                                               ...   
316384  서민 “조국, 자녀 관련 거짓말 끔찍이 싫어 고소… 아들 입대 두 달 남아” 압박   
316385              ‘김신조 사건’ 후 52년… 북악산 철문 직접 연 文 대통령   
316386          “주례 부탁드려도 될까요” 예비부부 부탁에 정세

In [9]:
tf_vect = CountVectorizer(tokenizer=getNVM_lemma, min_df=2)
dtm = tf_vect.fit_transform(docs['text'])

RuntimeError: 

In [6]:
vocab = dict()
for idx, word in enumerate(tf_vect.get_feature_names()):
    vocab[word] = dtm.getcol(idx).sum()  
words = sorted(vocab.items(), key = lambda x:x[1], reverse = True)

max = 20
plt.bar(range(max), [i[1] for i in words[:max]])
plt.title('Frequency Top 20')
plt.xlabel('단어')
ax = plt.subplot()
ax.set_xticks(range(max))
ax.set_xticklabels([i[0] for i in words[:max]], rotation = 30)
plt.show()

NameError: name 'tf_vect' is not defined